In [16]:
%matplotlib inline
from __future__ import division
import scipy.io as sio
import numpy as np
import math
import matplotlib.pyplot as plt
data = sio.loadmat('../data/programming/hw1progde.mat')
test_data = data['x_te']
training_data = data['x_tr']
h = 0.01
print(math.pi)
## 10 different measures, whcih one isbest? training or validation

3.14159265359


In [21]:
def K(x, XI, h, kernel_type='gaussian'):
    """
    Parameters
    ----------
    x: Point location
    XI: Vector of training data
    """
    if kernel_type == 'gaussian':
        s=0
        l=0
        for xi in XI:
            u = (x-xi)/h
            l+=1
            e = (1/math.sqrt(2*math.pi))*math.exp(-u*u/2)
            s+=e
        return s/(l*h)
    elif kernel_type == 'epanechnikov':
        s=0
        l=0
        for xi in XI:
            u = (x-xi)/h
            l+=1
            if abs(u)<=1:
                s+=(3/4)*(1-u^2)
        return s/(l*h)
    elif kernel_type == 'histogram':
        s=0
        l=0
        for xi in XI:
            u = (x-xi)/h
            l=0
            if abs(u)<-1:
                s+=0.5
        return s/(l*h)
        
gaussian_r = [K(x[0],training_data, 1, 'gaussian') for x in test_data]

hist_r = [K(x[0],training_data, 1, 'histogram') for x in test_data]

ea_r = [K(x[0],training_data, 1, 'epanechnikov') for x in test_data]


AttributeError: 'module' object has no attribute 'abs'

In [ ]:
plt.plot(gaussian_r)

In [ ]:
plt.plot(test_data)

In [ ]:
plt.hist(test_data, bins=40, fc='#AAAAFF')#, normed=True)


In [ ]:
plt.plot(ea_r)

In [ ]:
plt.plot(hist_r)

## Gaussian Kernel:
$$
K(u) = \frac{1}{\sqrt{2\pi}} e^{-\frac{u^2}{2}} \forall u \in \mathbb{R}
$$

## Epanechnikov kernel

$$
K(u) = \begin{cases}
\frac{3}{4}[1-u^2] & |u| \in [0,1],\\
0 & otherwise
\end{cases}
$$

## Histogram
$$
K(u) = \begin{cases}
\frac{1}{2}  & |u| \leq 1\\
0 & otherwise \\
\end{cases}
$$